In [29]:
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd

marathon_2015_2017 = pd.read_csv("./marathon_2015_2017.csv")

In [30]:
marathon_2015_2017['M/F'] = marathon_2015_2017['M/F'].map({'M': 1, 'F':0})
# 2015-2016 데이터와 2017 데이토로 분리
marathon_2015_2016 = marathon_2015_2017[ marathon_2015_2017['Year'] != 2017]
marathon_2017 = marathon_2015_2017[ marathon_2015_2017['Year'] == 2017]

# 별도의 데이터 프레임으로 필요한 필드만 취하여 저장
df_2015_2016 = pd.DataFrame(marathon_2015_2016, \
           columns=['M/F', 'Age', 'Pace', '10K', '20K', '30K', 'Official Time'])
df_2017 = pd.DataFrame(marathon_2017, \
           columns=['M/F', 'Age', 'Pace', '10K', '20K', '30K', 'Official Time'])

# 각 데이터 프레임을 리스트로 변환
record_2015_2016 = df_2015_2016.values.tolist()
record_2017 = df_2017.values.tolist()

# train과 test / X와 y로 변환
x_train = [ r[0:3] for r in record_2015_2016]
y_train = [ r[3:7] for r in record_2015_2016]
x_test = [ r[0:3] for r in record_2017]
y_test = [ r[3:7] for r in record_2017]

gender_list = ['Female', 'Male']
fig = Figure( figsize=(10,4), dpi=100)
ax = fig.add_subplot(111)
ax.set_xlim(15,88) # 나이를 표시할 x축의 범위
ax.set_ylim(0,1300) # 페이스가 표시될 y축의 범위
ax.set_xlabel("Age : Age on race day")
ax.set_ylabel("Pace : Runner's overall minute per mile pace")
xdata, ydata = [], []

In [31]:
def histogram():
    t_a = int(t_aSpbox.get()) -1 # 순위 데이터는 0부터 1위로 표시
    runner = x_test[t_a]
    t_g = int(runner[0])
    t_a = int(runner[1])
    t_p = int(runner[2])
    if(t_g): gender_color = 'b'
    else: gender_color = 'r'
    gender_record = df_2017[df_2017['M/F'] == t_g]
    gender_age_record = gender_record[gender_record.Age == t_a]
    gender_age_record_list = gender_age_record.values.tolist()
    ax.plot(gender_record.Age, gender_record.Pace, '.', color=gender_color, alpha=0.3)
    ax.plot(t_a, t_p, 'yd')
    stat = gender_age_record['Pace'].describe()
    title = 'Gender : '+gender_list[t_g]+', Age : '+str(t_a)+', Pace : '+str(t_p)
    ax.set_title(title)
    ax.annotate('['+gender_list[t_g]+', '+str(t_a)+']', (75, 1200), fontsize=10)
    ax.annotate("%10s %7i"%('Count : ', stat[0]), (75, 1150), fontsize=10)
    ax.annotate("%10s %7.3f"%('25% : ', stat[1]), (75, 1100), fontsize=10)
    ax.annotate("%10s %7.3f"%('75% : ', stat[3]), (75, 1050), fontsize=10)
    ax.annotate("%10s %7.3f"%('75% : ', stat[5]), (75, 1000), fontsize=10)
    fig.canvas.draw()

In [32]:
def seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i%02i" % (hours, minutes, seconds)

In [35]:
def learning():
    import tensorflow as tf
    import numpy as np
    t_a = int(t_aSpbox.get()) -1
    runner = x_test[t_a] # 예측 및 테스트 데이터 x_test -2017년 데이터
    t_g = int(runner[0]) # 성별
    t_y = int(runner[1]) # 나이
    t_p = int(runner[2]) # 페이스
    t_t = int(t_tSpbox.get())
    t_r = float(t_rSpbox.get())
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(4, input_shape=(3,))) # 출력개수 4
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    model.compile(loss='mse', optimizer=sgd)
    model.summary()
    # x_train, y_train 학습 - 2015~2016년 데이터
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
    
    log_ScrolledText.insert(END, '\nGender:'+gender_list[t_g]+'\, Age:'+str(t_y)+\
                           ', Pace:'+str(t_p)+'\n', 'TITLE')
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n', 'HEADER')
    log_ScrolledText.insert(END, '%20s %20s' % ('Step', 'Cost')+'\n\n')
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            log_ScrolledText.insert(END, "%20i %20.5f" % (step, cost_val)+'\n')
            
    winner = [t_g, t_y, t_p]
    time = model.predict(np.array([winner]))
    log_ScrolledText.insert(END, "%20s" % ('\n\nRecords Prediction\n\n'), \
                            'HEADER')
    log_ScrolledText.insert(END, "%20s %30s %30s %20s" % ('Distance(km)', \
                        'Real record', 'ML Prediction', 'Variation(Second)')+'\n\n')
    distance = [10.,20.,30., 42.195]
    for i in range(len(time[0])):  # 예측결과 개수만큼 반복
        dist_time = int(time[0][i])
        real_time = seconds_to_hhmmss(y_test[t_a][i])+'('+str(y_test[t_a][i])+')'
        ml_time = seconds_to_hhmmss(dist_time) + '('+str(dist_time)+')'
        variation = y_test[t_a][i] - dist_time
        log_ScrolledText.insert(END, "%20.3fs %30s %30s %20.3f" % (distance[i], \
                                        real_time, ml_time, variation)+'\n')

In [36]:
# 윈도우 생성
main = Tk()
main.title("Multi Variable Output Linear Regression")
main.geometry()

#윈도우 제목 라벨
label = Label(main, text='Multi Variable Output Linear Regression')
label.config(font=("굴림", 18))
label.grid(row=0, column=0, columnspan=6)
# 순위를 지정할 스핀박스 & 라벨
t_aVal = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal, from_=0, to=len(x_test), increment=1)
t_aSpbox.grid(row=1, column=1)
t_aLabel=Label(main, text='Rank of runner : ')
t_aLabel.grid(row=1, column=0)
# 학습 횟수를 지정할 스핀박스 & 라벨
t_tVal = IntVar(value=2000)
t_tSpbox = Spinbox(main, textvariable=t_tVal, from_=0, to=100000, increment=1000)
t_tSpbox.grid(row=1, column=3)
t_tLabel=Label(main, text='Number of train : ')
t_tLabel.grid(row=1, column=2)
# 러닝 레이트를 지정할 스핀박스 & 라벨
t_rVal = DoubleVar(value=1e-6)
t_rSpbox = Spinbox(main, textvariable=t_rVal, from_=0, to=1, increment=1e-6)
t_rSpbox.grid(row=1, column=5)
t_rLabel=Label(main, text='Learning rate : ')
t_rLabel.grid(row=1, column=4)

# 그래프 그리는 버튼과 예측버튼버튼
btn1 = Button(main, text="Histogram", height=2,command=lambda:histogram())
btn1.grid(row=2, column=0, columnspan=3, sticky=(W,E))
btn2 = Button(main, text="Prediction", height=2,command=lambda:learning())
btn2.grid(row=2, column=3, columnspan=3, sticky=(W,E))

# 스크롤텍스트와 스타일 설정
grad_canvas = FigureCanvasTkAgg(fig, main)
grad_canvas.get_tk_widget().grid(row=3, column=0, columnspan=6)

log_ScrolledText = tkst.ScrolledText(main,height=15)
log_ScrolledText.grid(row=4, column=0, columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')
log_ScrolledText.tag_config('RESULT', foreground='blue', font=("굴림", 12))
log_ScrolledText.tag_config('HEADER', foreground='black', font=("굴림", 14))
log_ScrolledText.tag_config('TITLE', foreground='red', font=("굴림", 18))

main.mainloop()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
1664/1664 [==============================] - 1s 473us/step - loss: 1446705.9564
Epoch 2/2000
1664/1664 [==============================] - 1s 464us/step - loss: 108239.8877
Epoch 3/2000
1664/1664 [==============================] - 1s 470us/step - loss: 102139.5342
Epoch 4/2000
1664/1664 [==============================] - 1s 475us/step - loss: 102588.6098
Epoch 5/2000
1664/1664 [==============================] - 1s 498us/step - loss: 119373.1528
Epoch 6/2000
1664/1664 [==============================] - 1s 485us/step - loss: 113863.5859
Epoch 7/2000
1664/1664 [==============================] - 1s 477us/step - loss: 110535.4455
Epoch 8

1664/1664 [==============================] - 1s 458us/step - loss: 124673.9483
Epoch 85/2000
1664/1664 [==============================] - 1s 460us/step - loss: 116817.3611
Epoch 86/2000
1664/1664 [==============================] - 1s 458us/step - loss: 113004.7618
Epoch 87/2000
1664/1664 [==============================] - 1s 468us/step - loss: 101027.1949
Epoch 88/2000
1664/1664 [==============================] - 1s 463us/step - loss: 112544.0040
Epoch 89/2000
1664/1664 [==============================] - 1s 459us/step - loss: 111062.1643
Epoch 90/2000
1664/1664 [==============================] - 1s 456us/step - loss: 116487.4587
Epoch 91/2000
1664/1664 [==============================] - 1s 458us/step - loss: 111579.7273
Epoch 92/2000
1664/1664 [==============================] - 1s 465us/step - loss: 103643.8218
Epoch 93/2000
1664/1664 [==============================] - 1s 463us/step - loss: 115594.7198
Epoch 94/2000
1664/1664 [==============================] - 1s 458us/step - loss: 119

1664/1664 [==============================] - 1s 464us/step - loss: 102188.9335
Epoch 172/2000
1664/1664 [==============================] - 1s 454us/step - loss: 106787.4938
Epoch 173/2000
1664/1664 [==============================] - 1s 454us/step - loss: 109141.4294
Epoch 174/2000
1664/1664 [==============================] - 1s 452us/step - loss: 115026.0120
Epoch 175/2000
1664/1664 [==============================] - 1s 461us/step - loss: 118353.3498
Epoch 176/2000
1664/1664 [==============================] - 1s 457us/step - loss: 113736.6829
Epoch 177/2000
1664/1664 [==============================] - 1s 462us/step - loss: 113466.7829
Epoch 178/2000
1664/1664 [==============================] - 1s 470us/step - loss: 109068.0326
Epoch 179/2000
1664/1664 [==============================] - 1s 473us/step - loss: 120208.1155
Epoch 180/2000
1664/1664 [==============================] - 1s 461us/step - loss: 111035.9688
Epoch 181/2000
1664/1664 [==============================] - 1s 468us/step -

1664/1664 [==============================] - 1s 468us/step - loss: 106849.1775
Epoch 259/2000
1664/1664 [==============================] - 1s 461us/step - loss: 107683.3712
Epoch 260/2000
1664/1664 [==============================] - 1s 460us/step - loss: 100486.1017
Epoch 261/2000
1664/1664 [==============================] - 1s 469us/step - loss: 107288.7641
Epoch 262/2000
1664/1664 [==============================] - 1s 454us/step - loss: 100889.7739
Epoch 263/2000
1664/1664 [==============================] - 1s 465us/step - loss: 104451.8293
Epoch 264/2000
1664/1664 [==============================] - 1s 468us/step - loss: 117500.0432
Epoch 265/2000
1664/1664 [==============================] - 1s 453us/step - loss: 115178.8828
Epoch 266/2000
1664/1664 [==============================] - 1s 460us/step - loss: 100666.4163
Epoch 267/2000
1664/1664 [==============================] - 1s 467us/step - loss: 109119.5173
Epoch 268/2000
1664/1664 [==============================] - 1s 456us/step -

1664/1664 [==============================] - 1s 454us/step - loss: 108607.9677
Epoch 346/2000
1664/1664 [==============================] - 1s 466us/step - loss: 108624.6650
Epoch 347/2000
1664/1664 [==============================] - 1s 459us/step - loss: 105278.5713
Epoch 348/2000
1664/1664 [==============================] - 1s 451us/step - loss: 111286.5777
Epoch 349/2000
1664/1664 [==============================] - 1s 462us/step - loss: 106453.4537
Epoch 350/2000
1664/1664 [==============================] - 1s 458us/step - loss: 108762.8071
Epoch 351/2000
1664/1664 [==============================] - 1s 447us/step - loss: 102382.2024
Epoch 352/2000
1664/1664 [==============================] - 1s 456us/step - loss: 109622.8760
Epoch 353/2000
1664/1664 [==============================] - 1s 462us/step - loss: 108223.0081
Epoch 354/2000
1664/1664 [==============================] - 1s 457us/step - loss: 107624.7485
Epoch 355/2000
1664/1664 [==============================] - 1s 459us/step -

...

1664/1664 [==============================] - 1s 454us/step - loss: 95950.0121
Epoch 1644/2000
1664/1664 [==============================] - 1s 443us/step - loss: 102318.8793
Epoch 1645/2000
1664/1664 [==============================] - 1s 446us/step - loss: 104304.3568
Epoch 1646/2000
1664/1664 [==============================] - 1s 449us/step - loss: 101450.1247
Epoch 1647/2000
1664/1664 [==============================] - 1s 439us/step - loss: 106775.3392
Epoch 1648/2000
1664/1664 [==============================] - 1s 449us/step - loss: 101573.5044
Epoch 1649/2000
1664/1664 [==============================] - 1s 454us/step - loss: 104738.4173
Epoch 1650/2000
1664/1664 [==============================] - 1s 450us/step - loss: 100678.8716
Epoch 1651/2000
1664/1664 [==============================] - 1s 459us/step - loss: 119909.4026
Epoch 1652/2000
1664/1664 [==============================] - 1s 445us/step - loss: 117833.3495
Epoch 1653/2000
1664/1664 [==============================] - 1s 446

1664/1664 [==============================] - 1s 438us/step - loss: 102536.7171
Epoch 1730/2000
1664/1664 [==============================] - 1s 447us/step - loss: 115772.6285
Epoch 1731/2000
1664/1664 [==============================] - 1s 468us/step - loss: 103868.3151
Epoch 1732/2000
1664/1664 [==============================] - 1s 445us/step - loss: 121345.8520
Epoch 1733/2000
1664/1664 [==============================] - 1s 441us/step - loss: 101140.8423
Epoch 1734/2000
1664/1664 [==============================] - 1s 448us/step - loss: 121599.3435
Epoch 1735/2000
1664/1664 [==============================] - 1s 448us/step - loss: 107195.8606
Epoch 1736/2000
1664/1664 [==============================] - 1s 440us/step - loss: 106663.0435
Epoch 1737/2000
1664/1664 [==============================] - 1s 438us/step - loss: 110057.2738
Epoch 1738/2000
1664/1664 [==============================] - 1s 439us/step - loss: 100961.7347
Epoch 1739/2000
1664/1664 [==============================] - 1s 44

1664/1664 [==============================] - 1s 449us/step - loss: 118605.8175
Epoch 1816/2000
1664/1664 [==============================] - 1s 453us/step - loss: 105619.5261
Epoch 1817/2000
1664/1664 [==============================] - 1s 446us/step - loss: 103252.5214
Epoch 1818/2000
1664/1664 [==============================] - 1s 450us/step - loss: 111398.7316
Epoch 1819/2000
1664/1664 [==============================] - 1s 437us/step - loss: 101555.9482
Epoch 1820/2000
1664/1664 [==============================] - 1s 439us/step - loss: 113223.3930
Epoch 1821/2000
1664/1664 [==============================] - 1s 448us/step - loss: 114628.6522
Epoch 1822/2000
1664/1664 [==============================] - 1s 430us/step - loss: 110294.5928
Epoch 1823/2000
1664/1664 [==============================] - 1s 442us/step - loss: 115532.2342
Epoch 1824/2000
1664/1664 [==============================] - 1s 445us/step - loss: 118058.4026
Epoch 1825/2000
1664/1664 [==============================] - 1s 44

1664/1664 [==============================] - 1s 449us/step - loss: 97062.1024
Epoch 1902/2000
1664/1664 [==============================] - 1s 446us/step - loss: 105334.8277
Epoch 1903/2000
1664/1664 [==============================] - 1s 439us/step - loss: 103685.7430
Epoch 1904/2000
1664/1664 [==============================] - 1s 444us/step - loss: 115698.6345
Epoch 1905/2000
1664/1664 [==============================] - 1s 442us/step - loss: 112845.1613
Epoch 1906/2000
1664/1664 [==============================] - 1s 445us/step - loss: 112470.5292
Epoch 1907/2000
1664/1664 [==============================] - 1s 444us/step - loss: 114377.5920
Epoch 1908/2000
1664/1664 [==============================] - 1s 436us/step - loss: 106081.6561
Epoch 1909/2000
1664/1664 [==============================] - 1s 451us/step - loss: 135984.8339
Epoch 1910/2000
1664/1664 [==============================] - 1s 440us/step - loss: 109076.4866
Epoch 1911/2000
1664/1664 [==============================] - 1s 451

1664/1664 [==============================] - 1s 449us/step - loss: 118402.0544
Epoch 1988/2000
1664/1664 [==============================] - 1s 441us/step - loss: 112194.1091
Epoch 1989/2000
1664/1664 [==============================] - 1s 442us/step - loss: 105692.1509
Epoch 1990/2000
1664/1664 [==============================] - 1s 453us/step - loss: 109261.7112
Epoch 1991/2000
1664/1664 [==============================] - 1s 446us/step - loss: 104804.0006
Epoch 1992/2000
1664/1664 [==============================] - 1s 441us/step - loss: 106410.7328
Epoch 1993/2000
1664/1664 [==============================] - 1s 451us/step - loss: 104521.6113
Epoch 1994/2000
1664/1664 [==============================] - 1s 440us/step - loss: 104028.7224
Epoch 1995/2000
1664/1664 [==============================] - 1s 440us/step - loss: 104795.4479
Epoch 1996/2000
1664/1664 [==============================] - 1s 442us/step - loss: 108384.5854
Epoch 1997/2000
1664/1664 [==============================] - 1s 44